<a href="https://colab.research.google.com/github/akashmathur-2212/LLMs-playground/blob/main/LlamaIndex-applications%20/llama2-multi-documents-chatbot/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qqq langchain llama-index llama-hub openai accelerate==0.21.0 bitsandbytes==0.40.2 transformers sentence_transformers InstructorEmbedding chromadb pypdf pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.1/905.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB

In [2]:
import torch
import logging
import sys

# llama_index
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext, load_index_from_storage, ServiceContext
from langchain.vectorstores import Chroma

from transformers import AutoTokenizer, TextStreamer, pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer


# llama_index
from llama_index.response.notebook_utils import display_response
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.response_synthesizers import get_response_synthesizer
from llama_index import SummaryIndex
from llama_index.text_splitter import SentenceSplitter
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.response.pprint_utils import pprint_response
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM

# chromadb
import chromadb

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# ChatBot using LlamaIndex

# Data Connectors

In [3]:
# Earning reports of 3 banks
!ls './sample_data/pdfs'

axis_report.pdf  hdfc_report.pdf  icici_report.pdf


In [4]:
# Load data
documents = SimpleDirectoryReader('./sample_data/pdfs').load_data()
len(documents)

192

# Models

## LLM

In [7]:
from google.colab import userdata

# huggingface api token for downloading llama2
hf_token = userdata.get('hf_token')

In [8]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    query_wrapper_prompt=PromptTemplate("<s> [INST] {query_str} [/INST] "),
    context_window=3900,
    model_kwargs={"token": hf_token, "quantization_config": quantization_config},
    tokenizer_kwargs={"token": hf_token},
    device_map="auto",
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Embedding Model

In [9]:
embed_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


# Defining the Service Context

In [10]:
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20) # using the default chunk_# values as they work just fine

# set context window
context_window = 4096
# set number of output tokens
num_output = 256

service_context = ServiceContext.from_defaults(llm=llm,
                                               embed_model=embed_model,
                                               text_splitter=text_splitter,
                                              #  context_window=context_window,
                                              #  num_output=num_output,
                                               )

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Index Setup

In [11]:
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [12]:
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

# Storing

In [13]:
# # To make ephemeral client that is a short lasting client or an in-memory client
# db = chromadb.EphemeralClient()

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("bank_earnings_database")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
vector_index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context
)

# Querying

### Compact (default)

In [16]:
# create a query engine and query

query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What is the Q2FY24 Net revenue for HDFC Bank?")

pprint_response(response, show_source=True)

Final Response: Based on the provided context information, the Q2FY24
net revenue for HDFC Bank is ₹380.9 billion.  According to the
earnings presentation, the standalone Indian GAAP figures for HDFC
Bank's net revenue for Q2FY24 are:  * Q1FY24: ₹328.3 billion * Q2FY24:
₹380.9 billion  Therefore, the Q2FY24 net revenue for HDFC Bank is
₹380.9 billion.
______________________________________________________________________
Source Node 1/2
Node ID: 95d1e74f-3025-4e5c-b1a5-30235c01a94c
Similarity: 0.8623806017200899
Text: Q2FY24 Earnings Presentation  6 HDFC Limited merged with HDFC
Bank effective July 1, 2023. Prior period numbers are not
comparable.STANDALONE Income Statement P&L ( ₹ bn) Q1FY24 Q2FY24 QoQ
Net interest income 236.0 273.9 16.0% Non-interest income 92.3 107.1
16.0% Net revenue 328.3 380.9 16.0% Operating expenses 140.6 154.0
9.5% PPOP 187.7 226.9 ...
______________________________________________________________________
Source Node 2/2
Node ID: 09fae70d-5583-4730-b505-dec47

In [17]:
response.source_nodes[0]

NodeWithScore(node=TextNode(id_='95d1e74f-3025-4e5c-b1a5-30235c01a94c', embedding=None, metadata={'page_label': '6', 'file_name': 'hdfc_report.pdf', 'file_path': 'sample_data/pdfs/hdfc_report.pdf', 'file_type': 'application/pdf', 'file_size': 379027, 'creation_date': '2023-11-28', 'last_modified_date': '2023-11-28', 'last_accessed_date': '2023-11-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='29c134d9-81bb-44eb-b09e-508776416242', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '6', 'file_name': 'hdfc_report.pdf', 'file_path': 'sample_data/pdfs/hdfc_report.pdf', 'file_type': 'application/pdf', 'file_size': 379027, 'creation_date': '2023-11-28', 'last_modified_date': '2023-11-28', 'last_

In [18]:
response.source_nodes[1]

NodeWithScore(node=TextNode(id_='09fae70d-5583-4730-b505-dec47bcf7a8f', embedding=None, metadata={'page_label': '5', 'file_name': 'hdfc_report.pdf', 'file_path': 'sample_data/pdfs/hdfc_report.pdf', 'file_type': 'application/pdf', 'file_size': 379027, 'creation_date': '2023-11-28', 'last_modified_date': '2023-11-28', 'last_accessed_date': '2023-11-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ab8b13e6-f2d0-49c3-b27d-ebe664f877f0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '5', 'file_name': 'hdfc_report.pdf', 'file_path': 'sample_data/pdfs/hdfc_report.pdf', 'file_type': 'application/pdf', 'file_size': 379027, 'creation_date': '2023-11-28', 'last_modified_date': '2023-11-28', 'last_

In [19]:
response = query_engine.query("What is the Q2FY24 Consolidated Net revenue for HDFC Bank?")

pprint_response(response, show_source=True)

Final Response: Based on the context information provided, the Q2FY24
Consolidated Net revenue for HDFC Bank is ₹663.2 billion.
______________________________________________________________________
Source Node 1/2
Node ID: 95d1e74f-3025-4e5c-b1a5-30235c01a94c
Similarity: 0.8696262009156825
Text: Q2FY24 Earnings Presentation  6 HDFC Limited merged with HDFC
Bank effective July 1, 2023. Prior period numbers are not
comparable.STANDALONE Income Statement P&L ( ₹ bn) Q1FY24 Q2FY24 QoQ
Net interest income 236.0 273.9 16.0% Non-interest income 92.3 107.1
16.0% Net revenue 328.3 380.9 16.0% Operating expenses 140.6 154.0
9.5% PPOP 187.7 226.9 ...
______________________________________________________________________
Source Node 2/2
Node ID: cdc62cb1-609d-481f-bfad-c9bffcae8e4f
Similarity: 0.8692582662044016
Text: Q2FY24 Earnings Presentation  4 HDFC Limited merged with HDFC
Bank effective July 1, 2023. Prior period numbers are not
comparable.CONSOLIDATED Consolidated Income Statement P&L ₹ b

In [20]:
response = query_engine.query("How much is the Gross NPA in Q2FY24 for HDFC Bank?")

display_response(response)

**`Final Response:`** Based on the information provided in the context, the Gross NPA in Q2FY24 for HDFC Bank is 1.34%. This can be calculated by looking at the last line of the table under the "Key Financial Parameters for Q2FY24" section, which states "Gross NPA 1.34%".

In [21]:
response = query_engine.query("How does the HDFC Life Insurance performance looks like in Q2FY24?")

display_response(response)

**`Final Response:`** Based on the provided context information, the HDFC Life Insurance performance in Q2FY24 looks like this:

* Premium Income: ₹ 149 billion, up by 12% YoY
* AUM: ₹ 2.6 trillion, up by 18% YoY
* Individual Weighted Received premium market share: 15.7% for H1FY24
* New Business Premium: ₹ 71 billion, up by 11% YoY
* Value of new business: ₹ 8 billion, up by 4% YoY
* PAT: ₹ 3.8 billion, up by 15% YoY
* Solvency Ratio: 194% at September 30, 2023

Overall, the HDFC Life Insurance performance in Q2FY24 shows a positive trend, with growth in premium income, AUM, and new business premium, and an increase in profitability. The solvency ratio also improved to 194%,

In [22]:
response = query_engine.query("How does the HDFC Life Insurance performance looks like in Q2FY24?")

display_response(response)

**`Final Response:`** Based on the context information provided, the HDFC Life Insurance performance in Q2FY24 looks like this:

* Premium Income: ₹ 149 billion, up by 12% YoY
* AUM: ₹ 2.6 trillion, up by 18% YoY
* Individual Weighted Received premium market share: 15.7% for H1FY24
* New Business Premium: ₹ 71 billion, up by 11% YoY
* Value of new business: ₹ 8 billion, up by 4% YoY
* PAT: ₹ 3.8 billion, up by 15% YoY
* Solvency Ratio: 194% at September 30, 2023

Therefore, the HDFC Life Insurance performance in Q2FY24 shows an increase in premium income, AUM, and new business premium, along with a higher solvency ratio.

In [23]:
response = query_engine.query("What steps HDFC Bank is taking towards sustainablity?")

display_response(response)

**`Final Response:`** Based on the context information provided, HDFC Bank is taking several steps towards sustainability:

1. Carbon neutrality: HDFC Bank aims to be carbon neutral by FY32.
2. Green branches: The bank is committed to having all new branches certified as green.
3. Renewable energy: HDFC Bank has financed 6,110 MW of renewable energy capacity as of March 31, 2023.
4. Green portfolio: The bank is focusing on increasing its green portfolio.
5. Responsible banking: HDFC Bank is committed to leading responsibly, taking banking to the unbanked, supporting businesses, enabling smart banking, and empowering communities.
6. CSR initiatives: The bank's CSR initiatives have cumulatively impacted over 99.3 million beneficiaries.
7. People and work culture: HDFC Bank is committed to nurturing, caring, and collaborating with its employees.
8. Governance: The bank is committed to principles of independence, accountability, responsibility, transparency

### Refine

In [24]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("Can you compare the Gross NPA and Net NPA of HDFC, ICICI and Axis Bank in the latest quarter?")

display_response(response)

**`Final Response:`** Based on the new context provided, we can compare the Gross NPA and Net NPA of HDFC, ICICI, and Axis Bank in the latest quarter.

According to the information provided in the Axis Bank report, the Gross NPA of Axis Bank as of September 2023 was `118 billion, which is 0.36% of its total assets. The Net NPA of Axis Bank was `118 billion, which is 1.32% of its total assets.

Similarly, the Gross NPA of HDFC as of September 2023 was `43 billion, which is 0.34% of its total assets. The Net NPA of HDFC was `42 billion, which is 0.31% of its total assets.

The Gross NPA of ICICI as of September 2023 was `37 billion, which is 0.35% of its total assets. The Net NPA of ICICI was `36 billion, which is 0.34% of its total assets.

Therefore,

In [25]:
query_engine_refined = vector_index.as_query_engine(response_mode="refine")

response = query_engine_refined.query("How does the HDFC Life Insurance performance looks like in Q2FY24?")

display_response(response)

**`Final Response:`** Based on the new context provided, HDFC Life Insurance's performance in Q2FY24 looks even more promising. As mentioned in the earnings presentation, HDFC ERGO General Insurance, a subsidiary of HDFC Limited, which merged with HDFC Bank effective July 1, 2023, has shown impressive growth in Q2FY24.

Key highlights from the Q2FY24 performance of HDFC ERGO General Insurance include:

* No. of policies issued: 2.7 million, up by 14% YoY
* Gross written premium: ₹ 53.5 billion, up by 8% YoY
* Net profit: ₹ 2.4 billion, up by 33% YoY
* Solvency Ratio: 191% at September 30, 2023

These numbers indicate that HDFC Life Insurance's performance in Q2FY24 was not only strong but also showed significant improvement compared to the same period last year, thanks to the merger

In [26]:
query_engine_refined = vector_index.as_query_engine(response_mode="refine")

response = query_engine_refined.query("What steps HDFC Bank is taking towards sustainablity?")

display_response(response)

**`Final Response:`** Based on the new context provided, here is a refined answer to the original query:

HDFC Bank is taking steps towards sustainability by:

1. Investing in green branches: HDFC Bank is committed to having all new branches certified as green, which suggests that they are prioritizing sustainability in their expansion efforts.
2. Financing renewable energy: HDFC Bank has financed 6,110 MW of renewable energy capacity as of March 31, 2023, which demonstrates their commitment to reducing their carbon footprint.
3. Supporting businesses and communities: HDFC Bank is focused on supporting businesses and enabling smart banking, as well as empowering communities through their CSR initiatives. They have cumulatively impacted over 99.3 million beneficiaries through their CSR efforts.
4. Leading responsibly: HDFC Bank is committed to leading responsibly, which includes nurturing, caring, and collaborating with stakeholders. They have also established a 2-tiered governance structure for diversity, equity, and inclusion at

### Tree Summarize

A Summary Index is a simpler form of Index best suited to queries where, as the name suggests, you are trying to generate a summary of the text in your Documents. It simply stores all of the Documents and returns all of them to your query engine.

A summarization query requires the LLM to iterate through many if not most documents in order to synthesize an answer. For instance, a summarization query could look like one of the following:
•	“What is a summary of this collection of text?”
•	“Give me a summary of person X’s experience with the company.”
In general, a summary index would be suited for this use case. A summary index by default goes through all the data.
Empirically, setting response_mode="tree_summarize" also leads to better summarization results.


In [44]:
%%time
query_engine_summarized = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine_summarized.query("Can you summarize the earnings report of Axis Bank?”")

display_response(response)

**`Final Response:`** Based on the information provided in the given context, here is a summary of the earnings report of Axis Bank:

* Axis Bank reported a net profit of ₹118 billion in Q2FY24, a growth of 26% YoY.
* The bank's total income increased by 14% YoY to ₹324 billion in Q2FY24.
* The bank's net interest income increased by 17% YoY to ₹144 billion in Q2FY24.
* The bank's provisions for bad loans increased by 31% YoY to ₹20 billion in Q2FY24.
* The bank's capital adequacy ratio (CAR) was 14.56%, while its common equity tier 1 (CET 1) ratio was 118 billion.
* The bank's credit card business saw a growth of 2% YoY, while its deposit growth was 16% YoY.
* The bank's advances grew by 114% Yo

CPU times: user 21.3 s, sys: 309 ms, total: 21.6 s
Wall time: 23.9 s


# Customize Your Retrieval Query

In [31]:
# configure retriever
retriever = VectorIndexRetriever(
    index=vector_index,
    similarity_top_k=6,
)

# configure node postprocessors
s_processor = SimilarityPostprocessor(similarity_cutoff=0.79)
k_processor = KeywordNodePostprocessor(
    exclude_keywords=["environmental"]
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(service_context=service_context)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[s_processor, k_processor],
    response_synthesizer=response_synthesizer
)

In [32]:
response = query_engine.query("What steps HDFC Bank is taking towards sustainablity?")

display_response(response)

**`Final Response:`** Based on the context information provided, it appears that HDFC Bank is taking steps towards sustainability by focusing on efficiency in growth and investing in new branches to expand its operations.

From the chart provided, it can be seen that HDFC Bank has been consistently improving its cost-to-income ratio, which suggests that the bank is becoming more efficient in its operations. This efficiency allows the bank to invest more in new branches, which in turn enables it to fast-track its growth.

Additionally, the bank's focus on benign credit enables it to invest in new branches without compromising on its credit quality. This suggests that the bank is taking a prudent approach to growth, rather than relying solely on aggressive lending practices that could potentially lead to a build-up of non-performing assets.

Overall, it appears that HDFC Bank is taking a well-rounded approach to sustainability by prioritizing efficiency, credit quality, and growth.

Use MMR **maximum marginal relevance**

Instead of ranking vectors purely by similarity, adds diversity to the documents by penalizing documents similar to ones that have already been found based on MMR . A lower mmr_treshold increases diversity.

In [ ]:
# query_engine = vector_index.as_query_engine(
#     vector_store_query_mode="mmr", vector_store_kwargs={"mmr_threshold": 0.2}
# )
# response_mmr = query_engine.query("What steps HDFC Bank is taking towards sustainablity?")

## Automated Metadata Extraction for Better Document Indexing & Retrieval

In many cases, especially with long documents, a chunk of text may lack the context necessary to disambiguate the chunk from other similar chunks of text. One method of addressing this is manually labelling each chunk in our dataset or knowledge base. However, this can be labour intensive and time consuming for a large number or continually updated set of documents.

To combat this, we use LLMs to extract certain contextual information relevant to the document to better help the retrieval and language models disambiguate similar-looking passages.

We do this through Metadata Extractor modules.

You can use LLMs to automate metadata extraction with our Metadata Extractor modules.


1. **SummaryExtractor** - automatically extracts a summary over a set of Nodes

2. **QuestionsAnsweredExtractor** - extracts a set of questions that each Node can answer

3. **TitleExtractor** - extracts a title over the context of each Node

4. **EntityExtractor** - extracts entities (i.e. names of places, people, things) mentioned in the content of each Node

Then you can chain the Metadata Extractors with the node parser

In [34]:
import nest_asyncio

nest_asyncio.apply()

from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
)

from llama_index.text_splitter import TokenTextSplitter
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode

In [35]:
node_parser = TokenTextSplitter(
    separator=" ", chunk_size=256, chunk_overlap=128
)


extractors_1 = [
    QuestionsAnsweredExtractor(
        questions=3, llm=llm, metadata_mode=MetadataMode.EMBED
    ),
]

extractors_2 = [
    SummaryExtractor(summaries=["prev", "self", "next"], llm=llm),
    QuestionsAnsweredExtractor(
        questions=3, llm=llm, metadata_mode=MetadataMode.EMBED
    ),
]

In [36]:
extractors_2 = [
    SummaryExtractor(summaries=["prev", "self", "next"], llm=llm),
    QuestionsAnsweredExtractor(
        questions=3, llm=llm, metadata_mode=MetadataMode.EMBED
    ),
]

In [37]:
orig_nodes = node_parser.get_nodes_from_documents(documents)

In [38]:
# take just the first 8 nodes for testing
nodes = orig_nodes[10:14]

In [39]:
print(nodes[3].get_content(metadata_mode="all"))

page_label: 3
file_name: axis_report.pdf
file_path: sample_data/pdfs/axis_report.pdf
file_type: application/pdf
file_size: 5580971
creation_date: 2023-11-28
last_modified_date: 2023-11-28
last_accessed_date: 2023-11-28

and 5% QOQ  
o Consolidated ROE | ROA at 18.67% | 1.83% , subsidiaries contributed 37 bps | 7 bps respectively  
• Strong  loan growth delivered across all business segments  
o Rural loans grew 24% YOY & 4% QOQ, Small Business Banking loans grew 42% YOY & 9% QOQ  
o Mid-Corporate ( MC) book grew 37% YOY , SME book grew 27% YOY & 9% QOQ  
o SBB + SME + MC mix at ₹1,88,573 crores  | 21% of loans, up over 680 bps in last 3 years   
• Retail term deposits gaining traction , steady growth in granular deposits   
o On a QAB1 basis total deposits grew 16% | 1% and retail term deposits grew 13% | 4% on YOY | QOQ basis  
o On QAB, SA grew 17% YOY


In [40]:
# Run metadata extractors
from llama_index.ingestion import IngestionPipeline

# process nodes with metadata extractors
pipeline = IngestionPipeline(transformations=[node_parser, *extractors_1])

nodes_1 = pipeline.run(nodes=nodes, in_place=False, show_progress=True)

Parsing nodes:   0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [01:30<00:00, 22.56s/it]


In [41]:
print(nodes_1[3].get_content(metadata_mode="all"))

[Excerpt from document]
page_label: 3
file_name: axis_report.pdf
file_path: sample_data/pdfs/axis_report.pdf
file_type: application/pdf
file_size: 5580971
creation_date: 2023-11-28
last_modified_date: 2023-11-28
last_accessed_date: 2023-11-28
questions_this_excerpt_can_answer: Based on the provided context, here are three questions that this context can provide specific answers to, and are unlikely to be found elsewhere:

1. What are the year-over-year (YOY) and quarter-over-quarter (QOQ) growth rates of rural loans and small business banking loans?

This context provides information on the growth rates of rural loans and small business banking loans, including the YOY and QOQ growth rates. Specifically, it states that rural loans grew 24% YOY and 4% QOQ, and small business banking loans grew 42% YOY and 9% QOQ.

2. What is the current mix of SBB, SME, and MC loans as a percentage of total loans?

This context provides information on the current mix of SBB, SME, and MC loans as a perce

In [ ]:
# # process nodes with metadata extractor
# pipeline = IngestionPipeline(transformations=[node_parser, *extractors_2])

# nodes_2 = pipeline.run(nodes=nodes, in_place=False, show_progress=True)

# print(nodes_2[3].get_content(metadata_mode="all"))

# Chat Engine

In [42]:
chat_engine = vector_index.as_chat_engine(chat_mode="condense_question")

In [45]:
response = chat_engine.chat("Can you provide important highlights of ICICI Bank's earnings report?")

In [46]:
print(response)

Based on the context information provided, the key highlights of ICICI Bank's latest earnings report are:

1. Profit before tax excluding treasury grew by 35.7% year-over-year (y-o-y) to ₹ 137.31 billion in Q2-2024.
2. Core operating profit grew by 21.7% y-o-y to ₹ 143.14 billion in Q2-2024.
3. Provisions of ₹ 5.83 billion in Q2-2024 (0.21% of average advances).
4. Period-end total deposits grew by 18.8% y-o-y and 4.5% q-o-q at September 30, 2023.
5. Period-end term deposits grew by 31.8% y-o-y and 9.2% q-o-q at September 30, 2023.
6. Average current account deposits increased by 14.0


In [47]:
response = chat_engine.chat("Can you highlight more details?")

In [48]:
print(response)

According to ICICI Bank's latest earnings report, the key highlights include:

* Profit before tax excluding treasury grew by 35.7% year-over-year to ₹ 137.31 billion in Q2-2024.
* Core operating profit grew by 21.7% year-over-year to ₹ 143.14 billion in Q2-2024.
* Provisions of ₹ 5.83 billion in Q2-2024 (0.21% of average advances).
* Period-end total deposits grew by 18.8% year-over-year and 4.5% quarter-over-quarter at September 30, 2023.
* Period-end term deposits grew by 31.8% year-over-year and 9.2% quarter-over-quarter at September 30, 2023.
* Average current account deposits increased by 14.0% year-over-year and 2.6% quarter-over-


In [49]:
chat_engine.reset()

In [50]:
response = chat_engine.chat("Do you want to share any other information which was not shared earlier?")

In [51]:
print(response)

Sure! Here's an example of how you could rewrite the follow-up message into a standalone question that captures all the relevant context from the conversation:

Standalone question: Can you provide any additional details or context about the "physical reach" and "digital prowess" mentioned in the previous message? How have you been extending your services to the "heart of Bharat"?


# Prompt Engineering

In [52]:
text_qa_template_str = (
    "You are a banking and finance expert and has access to all the earnings report.\n"
    "Always answer the query only using the provided context information, "
    "and not prior knowledge.\n"
    "Some rules to follow:\n"
    "1. Never directly reference the given context in your answer.\n"
    "2. Avoid statements like 'Based on the context, ...' or "
    "'The context information ...' or anything along "
    "those lines."
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Answer the question: {query_str}\n"
)

text_qa_template = PromptTemplate(text_qa_template_str)

In [53]:
response = vector_index.as_query_engine(
    text_qa_template = text_qa_template).query("How does the Capital and Liquidity Position of Axis Bank looks like in the recent quarter?")

In [55]:
display_response(response)

**`Final Response:`** Based on the provided context, the capital and liquidity position of Axis Bank appears to be strong in the recent quarter. The bank's CET-1 ratio has increased by 14.56% compared to the previous quarter, which is a positive sign. Additionally, the bank's liquidity coverage ratio (LCR) has improved significantly, with the bank holding excess SLR of `56,966 crores. This suggests that the bank has a strong capital position with adequate liquidity to meet its financial obligations. Furthermore, the bank's net accretion to CET-1 was 4.57% in the recent quarter, indicating a growth in its capital position. Overall, the capital and liquidity position of Axis Bank appears to be robust in the recent quarter.

In [57]:
pprint_response(response, show_source=True)

Final Response: Based on the provided context, the capital and
liquidity position of Axis Bank appears to be strong in the recent
quarter. The bank's CET-1 ratio has increased by 14.56% compared to
the previous quarter, which is a positive sign. Additionally, the
bank's liquidity coverage ratio (LCR) has improved significantly, with
the bank holding excess SLR of `56,966 crores. This suggests that the
bank has a strong capital position with adequate liquidity to meet its
financial obligations. Furthermore, the bank's net accretion to CET-1
was 4.57% in the recent quarter, indicating a growth in its capital
position. Overall, the capital and liquidity position of Axis Bank
appears to be robust in the recent quarter.
______________________________________________________________________
Source Node 1/2
Node ID: 21895e5d-2f5f-4ffc-b628-7eaa0c800890
Similarity: 0.8319586883542575
Text: Quarterly Results Q2FY242Cumulative provisions  (standard +
additional non -NPA)Axis Bank at a glance Axi

In [56]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='21895e5d-2f5f-4ffc-b628-7eaa0c800890', embedding=None, metadata={'page_label': '12', 'file_name': 'axis_report.pdf', 'file_path': 'sample_data/pdfs/axis_report.pdf', 'file_type': 'application/pdf', 'file_size': 5580971, 'creation_date': '2023-11-28', 'last_modified_date': '2023-11-28', 'last_accessed_date': '2023-11-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f8d72db0-d047-4cbd-b654-13e148f05388', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '12', 'file_name': 'axis_report.pdf', 'file_path': 'sample_data/pdfs/axis_report.pdf', 'file_type': 'application/pdf', 'file_size': 5580971, 'creation_date': '2023-11-28', 'last_modified_date': '2023-11-28', '

# End